In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install glove_python_binary
!pip install tqdm
# 텔레그램
!pip install python-telegram-bot --upgrade
!apt-get install telepot 
#

     |████████████████████████████████| 952kB 7.5MB/s 
     |████████████████████████████████| 491kB 9.6MB/s 
     |████████████████████████████████| 430kB 42.3MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible.
  Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1


Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package telepot


In [4]:
import numpy as np
from glove import Glove
from glove import Corpus

from tqdm.notebook import tqdm

#텔레그램
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler
import telegram

#

AttributeError: ignored

In [ ]:
!head /content/drive/MyDrive/joonhwan/corpus_mecab1.txt

In [ ]:
corpus_fname = '/content/drive/MyDrive/joonhwan/corpus_mecab.txt'
corpus_model = '/content/drive/MyDrive/joonhwan/corpus.model'
glove_model = '/content/drive/MyDrive/joonhwan/glove.model'


def make_corpus(load=False):

    if load:
        _corpus = Corpus.load(corpus_model)
    else:
        corpus = [sent.strip().split(" ") for sent in tqdm(open(corpus_fname, 'r',encoding = 'utf-8').readlines())]

        _corpus = Corpus()
        _corpus.fit(corpus, window=10)
        _corpus.save(corpus_model)

    return _corpus


def train(corpus=None, train=True):
    if train:
        print('Dict size: %s' % len(corpus.dictionary))
        print('Collocations: %s' % corpus.matrix.nnz)
        
        _glove = Glove(no_components=100, learning_rate=0.05)
        _glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=True)
        _glove.add_dictionary(corpus.dictionary)
        _glove.save(glove_model)
    else:
        print('Loading pre-trained GloVe model')
        _glove = Glove.load(glove_model)
    return _glove

corpus = make_corpus(load=False)
glove = train(corpus, train=True)

np.savetxt('/content/drive/MyDrive/joonhwan/glove-vector.tsv', glove.word_vectors, delimiter='\t')



with open('/content/drive/MyDrive/joonhwan/glove-metadata.tsv', 'w', encoding='utf-8') as f:
   
    for key in glove.dictionary.keys():
        f.write(f"{key}\n")
print("학습 완료!")

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!pip install konlpy

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

In [ ]:


telegram_token = '1702844077:AAHGt-dORzT8IMbDqBxLIx510IMkxVjoXN8' # 

bot = telegram.Bot(token = telegram_token)
updates = bot.getUpdates()
#따라하기

from konlpy.tag import Mecab 
    
def get_message(update, context):
  mecab = Mecab() 
  text = update.message.text
  print(text)
  nouns = mecab.morphs(text) 

  #print(len(nouns[:]))
  print(nouns[:])
  
  for i in nouns:
    print(i)
    
    try:
      model_result1 = glove.most_similar(i,number=10)
      #print(model_result1)
      print(i)
      print('욕설감지')
      
      update.message.reply_text('욕설감지')
    except Exception as ex:
      print('정상문자')
        

# 형태소 분석기 다른거 찾아보기 쌍자음을 구별못함  
# 시발 검색시 시발럼아가 유사단어로 나와야하는데 안나옴
# 니애미입력시 니, 애미만 출력, 채팅 단어 그대로 출력가능하게 해야함
# 유사단어도비교하여 검출
# 벡터의 시각화 만들기

# 실시간 작동
updater = Updater(telegram_token, use_context=True)
message_handler = MessageHandler(Filters.text & (~Filters.command), get_message)
updater.dispatcher.add_handler(message_handler)
updater.start_polling(timeout=3, clean=True)
updater.idle()
#